In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.{DataFrame, SparkSession}
val spark = SparkSession.builder.appName("Mortgage_ETL").getOrCreate

spark = org.apache.spark.sql.SparkSession@5ac61b3a


org.apache.spark.sql.SparkSession@5ac61b3a

In [2]:
object ReadPerformanceCsv {
  def apply(spark: SparkSession, path: String): DataFrame = {
    val performanceSchema = StructType(Array(
      StructField("loan_id", LongType),
      StructField("monthly_reporting_period", StringType),
      StructField("servicer", StringType),
      StructField("interest_rate", DoubleType),
      StructField("current_actual_upb", DoubleType),
      StructField("loan_age", DoubleType),
      StructField("remaining_months_to_legal_maturity", DoubleType),
      StructField("adj_remaining_months_to_maturity", DoubleType),
      StructField("maturity_date", StringType),
      StructField("msa", DoubleType),
      StructField("current_loan_delinquency_status", IntegerType),
      StructField("mod_flag", StringType),
      StructField("zero_balance_code", StringType),
      StructField("zero_balance_effective_date", StringType),
      StructField("last_paid_installment_date", StringType),
      StructField("foreclosed_after", StringType),
      StructField("disposition_date", StringType),
      StructField("foreclosure_costs", DoubleType),
      StructField("prop_preservation_and_repair_costs", DoubleType),
      StructField("asset_recovery_costs", DoubleType),
      StructField("misc_holding_expenses", DoubleType),
      StructField("holding_taxes", DoubleType),
      StructField("net_sale_proceeds", DoubleType),
      StructField("credit_enhancement_proceeds", DoubleType),
      StructField("repurchase_make_whole_proceeds", StringType),
      StructField("other_foreclosure_proceeds", DoubleType),
      StructField("non_interest_bearing_upb", DoubleType),
      StructField("principal_forgiveness_upb", StringType),
      StructField("repurchase_make_whole_proceeds_flag", StringType),
      StructField("foreclosure_principal_write_off_amount", StringType),
      StructField("servicing_activity_indicator", StringType))
    )

    val udf = spark.udf.register("get_quarter", (path: String) => {
      path.split("\\.").head.split("_").last
    })

    spark.read.format("csv")
      .option("nullValue", "")
      .option("header", "false")
      .option("delimiter", "|")
      .option("parserLib", "univocity")
      .schema(performanceSchema)
      .load(path)
      .withColumn("quarter", udf(input_file_name()))
  }
}

object ReadAcquisitionCsv {
  def apply(spark: SparkSession, path: String): DataFrame = {
    val acquisitionSchema = StructType(Array(
      StructField("loan_id", LongType),
      StructField("orig_channel", StringType),
      StructField("seller_name", StringType),
      StructField("orig_interest_rate", DoubleType),
      StructField("orig_upb", IntegerType),
      StructField("orig_loan_term", IntegerType),
      StructField("orig_date", StringType),
      StructField("first_pay_date", StringType),
      StructField("orig_ltv", DoubleType),
      StructField("orig_cltv", DoubleType),
      StructField("num_borrowers", DoubleType),
      StructField("dti", DoubleType),
      StructField("borrower_credit_score", DoubleType),
      StructField("first_home_buyer", StringType),
      StructField("loan_purpose", StringType),
      StructField("property_type", StringType),
      StructField("num_units", IntegerType),
      StructField("occupancy_status", StringType),
      StructField("property_state", StringType),
      StructField("zip", IntegerType),
      StructField("mortgage_insurance_percent", DoubleType),
      StructField("product_type", StringType),
      StructField("coborrow_credit_score", DoubleType),
      StructField("mortgage_insurance_type", DoubleType),
      StructField("relocation_mortgage_indicator", StringType))
    )

    val udf = spark.udf.register("get_quarter", (path: String) => {
      path.split("\\.").head.split("_").last
    })

    spark.read.format("csv")
      .option("header", "false")
      .option("delimiter", "|")
      .schema(acquisitionSchema)
      .load(path)
      .withColumn("quarter", udf(input_file_name()))
  }
}

object NameMapping {
  def apply(): Map[String, String] = {
    Map(
      "WITMER FUNDING, LLC" -> "Witmer",
      "WELLS FARGO CREDIT RISK TRANSFER SECURITIES TRUST 2015" -> "Wells Fargo",
      "WELLS FARGO BANK,  NA" -> "Wells Fargo",
      "WELLS FARGO BANK, N.A." -> "Wells Fargo",
      "WELLS FARGO BANK, NA" -> "Wells Fargo",
      "USAA FEDERAL SAVINGS BANK" -> "USAA",
      "UNITED SHORE FINANCIAL SERVICES, LLC D\\/B\\/A UNITED WHOLESALE MORTGAGE" -> "United Seq(e",
      "U.S. BANK N.A." -> "US Bank",
      "SUNTRUST MORTGAGE INC." -> "Suntrust",
      "STONEGATE MORTGAGE CORPORATION" -> "Stonegate Mortgage",
      "STEARNS LENDING, LLC" -> "Stearns Lending",
      "STEARNS LENDING, INC." -> "Stearns Lending",
      "SIERRA PACIFIC MORTGAGE COMPANY, INC." -> "Sierra Pacific Mortgage",
      "REGIONS BANK" -> "Regions",
      "RBC MORTGAGE COMPANY" -> "RBC",
      "QUICKEN LOANS INC." -> "Quicken Loans",
      "PULTE MORTGAGE, L.L.C." -> "Pulte Mortgage",
      "PROVIDENT FUNDING ASSOCIATES, L.P." -> "Provident Funding",
      "PROSPECT MORTGAGE, LLC" -> "Prospect Mortgage",
      "PRINCIPAL RESIDENTIAL MORTGAGE CAPITAL RESOURCES, LLC" -> "Principal Residential",
      "PNC BANK, N.A." -> "PNC",
      "PMT CREDIT RISK TRANSFER TRUST 2015-2" -> "PennyMac",
      "PHH MORTGAGE CORPORATION" -> "PHH Mortgage",
      "PENNYMAC CORP." -> "PennyMac",
      "PACIFIC UNION FINANCIAL, LLC" -> "Other",
      "OTHER" -> "Other",
      "NYCB MORTGAGE COMPANY, LLC" -> "NYCB",
      "NEW YORK COMMUNITY BANK" -> "NYCB",
      "NETBANK FUNDING SERVICES" -> "Netbank",
      "NATIONSTAR MORTGAGE, LLC" -> "Nationstar Mortgage",
      "METLIFE BANK, NA" -> "Metlife",
      "LOANDEPOT.COM, LLC" -> "LoanDepot.com",
      "J.P. MORGAN MADISON AVENUE SECURITIES TRUST, SERIES 2015-1" -> "JP Morgan Chase",
      "J.P. MORGAN MADISON AVENUE SECURITIES TRUST, SERIES 2014-1" -> "JP Morgan Chase",
      "JPMORGAN CHASE BANK, NATIONAL ASSOCIATION" -> "JP Morgan Chase",
      "JPMORGAN CHASE BANK, NA" -> "JP Morgan Chase",
      "JP MORGAN CHASE BANK, NA" -> "JP Morgan Chase",
      "IRWIN MORTGAGE, CORPORATION" -> "Irwin Mortgage",
      "IMPAC MORTGAGE CORP." -> "Impac Mortgage",
      "HSBC BANK USA, NATIONAL ASSOCIATION" -> "HSBC",
      "HOMEWARD RESIDENTIAL, INC." -> "Homeward Mortgage",
      "HOMESTREET BANK" -> "Other",
      "HOMEBRIDGE FINANCIAL SERVICES, INC." -> "HomeBridge",
      "HARWOOD STREET FUNDING I, LLC" -> "Harwood Mortgage",
      "GUILD MORTGAGE COMPANY" -> "Guild Mortgage",
      "GMAC MORTGAGE, LLC (USAA FEDERAL SAVINGS BANK)" -> "GMAC",
      "GMAC MORTGAGE, LLC" -> "GMAC",
      "GMAC (USAA)" -> "GMAC",
      "FREMONT BANK" -> "Fremont Bank",
      "FREEDOM MORTGAGE CORP." -> "Freedom Mortgage",
      "FRANKLIN AMERICAN MORTGAGE COMPANY" -> "Franklin America",
      "FLEET NATIONAL BANK" -> "Fleet National",
      "FLAGSTAR CAPITAL MARKETS CORPORATION" -> "Flagstar Bank",
      "FLAGSTAR BANK, FSB" -> "Flagstar Bank",
      "FIRST TENNESSEE BANK NATIONAL ASSOCIATION" -> "Other",
      "FIFTH THIRD BANK" -> "Fifth Third Bank",
      "FEDERAL HOME LOAN BANK OF CHICAGO" -> "Fedral Home of Chicago",
      "FDIC, RECEIVER, INDYMAC FEDERAL BANK FSB" -> "FDIC",
      "DOWNEY SAVINGS AND LOAN ASSOCIATION, F.A." -> "Downey Mortgage",
      "DITECH FINANCIAL LLC" -> "Ditech",
      "CITIMORTGAGE, INC." -> "Citi",
      "CHICAGO MORTGAGE SOLUTIONS DBA INTERFIRST MORTGAGE COMPANY" -> "Chicago Mortgage",
      "CHICAGO MORTGAGE SOLUTIONS DBA INTERBANK MORTGAGE COMPANY" -> "Chicago Mortgage",
      "CHASE HOME FINANCE, LLC" -> "JP Morgan Chase",
      "CHASE HOME FINANCE FRANKLIN AMERICAN MORTGAGE COMPANY" -> "JP Morgan Chase",
      "CHASE HOME FINANCE (CIE 1)" -> "JP Morgan Chase",
      "CHASE HOME FINANCE" -> "JP Morgan Chase",
      "CASHCALL, INC." -> "CashCall",
      "CAPITAL ONE, NATIONAL ASSOCIATION" -> "Capital One",
      "CALIBER HOME LOANS, INC." -> "Caliber Funding",
      "BISHOPS GATE RESIDENTIAL MORTGAGE TRUST" -> "Bishops Gate Mortgage",
      "BANK OF AMERICA, N.A." -> "Bank of America",
      "AMTRUST BANK" -> "AmTrust",
      "AMERISAVE MORTGAGE CORPORATION" -> "Amerisave",
      "AMERIHOME MORTGAGE COMPANY, LLC" -> "AmeriHome Mortgage",
      "ALLY BANK" -> "Ally Bank",
      "ACADEMY MORTGAGE CORPORATION" -> "Academy Mortgage",
      "NO CASH-OUT REFINANCE" -> "OTHER REFINANCE",
      "REFINANCE - NOT SPECIFIED" -> "OTHER REFINANCE",
      "Other REFINANCE" -> "OTHER REFINANCE"
    )
  }
}

object CreatePerformanceDelinquency {
  def prepare(df: DataFrame): DataFrame = {
    df
      .withColumn("monthly_reporting_period", to_date(col("monthly_reporting_period"), "MM/dd/yyyy"))
      .withColumn("monthly_reporting_period_month", month(col("monthly_reporting_period")))
      .withColumn("monthly_reporting_period_year", year(col("monthly_reporting_period")))
      .withColumn("monthly_reporting_period_day", dayofmonth(col("monthly_reporting_period")))
      .withColumn("last_paid_installment_date", to_date(col("last_paid_installment_date"), "MM/dd/yyyy"))
      .withColumn("foreclosed_after", to_date(col("foreclosed_after"), "MM/dd/yyyy"))
      .withColumn("disposition_date", to_date(col("disposition_date"), "MM/dd/yyyy"))
      .withColumn("maturity_date", to_date(col("maturity_date"), "MM/yyyy"))
      .withColumn("zero_balance_effective_date", to_date(col("zero_balance_effective_date"), "MM/yyyy"))
      .withColumn("current_actual_upb", col("current_actual_upb"))
      .withColumn("current_loan_delinquency_status", col("current_loan_delinquency_status"))
  }

  def apply(spark: SparkSession, df: DataFrame): DataFrame = {
    import spark.implicits._

    val aggDF = df
      .select(
        col("quarter"),
        col("loan_id"),
        col("current_loan_delinquency_status"),
        when(col("current_loan_delinquency_status") >= 1, col("monthly_reporting_period")).alias("delinquency_30"),
        when(col("current_loan_delinquency_status") >= 3, col("monthly_reporting_period")).alias("delinquency_90"),
        when(col("current_loan_delinquency_status") >= 6, col("monthly_reporting_period")).alias("delinquency_180")
      )
      .groupBy("quarter", "loan_id")
      .agg(
        max("current_loan_delinquency_status").alias("delinquency_12"),
        min("delinquency_30").alias("delinquency_30"),
        min("delinquency_90").alias("delinquency_90"),
        min("delinquency_180").alias("delinquency_180")
      )
      .select(
        col("quarter"),
        col("loan_id"),
        (col("delinquency_12") >= 1).alias("ever_30"),
        (col("delinquency_12") >= 3).alias("ever_90"),
        (col("delinquency_12") >= 6).alias("ever_180"),
        col("delinquency_30"),
        col("delinquency_90"),
        col("delinquency_180")
      )

    val joinedDf = df
      .withColumnRenamed("monthly_reporting_period", "timestamp")
      .withColumnRenamed("monthly_reporting_period_month", "timestamp_month")
      .withColumnRenamed("monthly_reporting_period_year", "timestamp_year")
      .withColumnRenamed("current_loan_delinquency_status", "delinquency_12")
      .withColumnRenamed("current_actual_upb", "upb_12")
      .select("quarter", "loan_id", "timestamp", "delinquency_12", "upb_12", "timestamp_month", "timestamp_year")
      .join(aggDF, Seq("loan_id", "quarter"), "left_outer")

    // calculate the 12 month delinquency and upb values
    val months = 12
    val testDf = joinedDf
      .crossJoin(0.until(months).toDF("month_y"))
      .select(
        col("quarter"),
        floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000) / months).alias("josh_mody"),
        floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000 - col("month_y")) / months).alias("josh_mody_n"),
        col("ever_30"),
        col("ever_90"),
        col("ever_180"),
        col("delinquency_30"),
        col("delinquency_90"),
        col("delinquency_180"),
        col("loan_id"),
        col("month_y"),
        col("delinquency_12"),
        col("upb_12")
      )
      .groupBy("quarter", "loan_id", "josh_mody_n", "ever_30", "ever_90", "ever_180", "delinquency_30", "delinquency_90", "delinquency_180", "month_y")
      .agg(max("delinquency_12").alias("delinquency_12"), min("upb_12").alias("upb_12"))
      .withColumn("timestamp_year", floor((lit(24000) + (col("josh_mody_n") * lit(months)) + (col("month_y") - 1)) / lit(12)))
      .withColumn("timestamp_month_tmp", pmod((lit(24000) + (col("josh_mody_n") * lit(months)) + col("month_y")), lit(12)))
      .withColumn("timestamp_month", when(col("timestamp_month_tmp") === lit(0), lit(12)).otherwise(col("timestamp_month_tmp")))
      .withColumn("delinquency_12", ((col("delinquency_12") > 3).cast("int") + (col("upb_12") === 0).cast("int")).alias("delinquency_12"))
      .drop("timestamp_month_tmp", "josh_mody_n", "month_y")

    df.withColumnRenamed("monthly_reporting_period_month", "timestamp_month")
      .withColumnRenamed("monthly_reporting_period_year", "timestamp_year")
      .join(testDf, Seq("quarter", "loan_id", "timestamp_year", "timestamp_month"), "left").drop("timestamp_year", "timestamp_month")
  }
}


object CreateAcquisition {
  def apply(spark: SparkSession, df: DataFrame): DataFrame = {
    val names = NameMapping()
    val nameMapping = udf((name: String) => names.getOrElse(name, null))
    df
      .withColumn("old_name", col("seller_name"))
      .withColumn("seller_name", nameMapping(col("seller_name")))
      .withColumn("orig_date", to_date(col("orig_date"), "MM/yyyy"))
      .withColumn("first_pay_date", to_date(col("first_pay_date"), "MM/yyyy"))
  }
}


object CleanAcquisitionPrime {
  def apply(spark: SparkSession, perfDF: DataFrame, acqDF: DataFrame): DataFrame = {
    val perf = CreatePerformanceDelinquency(spark, perfDF)
    val acq = CreateAcquisition(spark, acqDF)

    perf.join(acq, Seq("loan_id", "quarter"), "left_outer").drop("quarter")
  }
}

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{FeatureHasher, StringIndexer, VectorAssembler}
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.DoubleType

object MortgageXgBoost {
  val catCols = List(
    "orig_channel",
    "first_home_buyer",
    "loan_purpose",
    "property_type",
    "occupancy_status",
    "property_state",
    "product_type",
    "relocation_mortgage_indicator",
    "seller_name",
    "mod_flag"
  )

  val numericCols = List(
    "orig_interest_rate",
    "orig_upb",
    "orig_loan_term",
    "orig_ltv",
    "orig_cltv",
    "num_borrowers",
    "dti",
    "borrower_credit_score",
    "num_units",
    "zip",
    "mortgage_insurance_percent",
    "current_loan_delinquency_status",
    "current_actual_upb",
    "interest_rate",
    "loan_age",
    "msa",
    "non_interest_bearing_upb",
    "delinquency_12"
  )

  val allCols: List[String] = catCols ++ numericCols

  def transform(df: DataFrame): (DataFrame, DataFrame) = {
    val featureDF = df.select(catCols.map(c => (md5(col(c)) % 100).alias(c)) ++ numericCols.map(c => col(c)): _*)
      .withColumn("delinquency_12", when(col("delinquency_12") > 0, 1).otherwise(0))
      .na.fill(0.0f)
    val Array(dtrain, dtest) = featureDF.randomSplit(Array(0.8, 0.2))
    (dtrain, dtest)
  }
}



defined object ReadPerformanceCsv
defined object ReadAcquisitionCsv
defined object NameMapping
defined object CreatePerformanceDelinquency
defined object CreateAcquisition
defined object CleanAcquisitionPrime
defined object MortgageXgBoost


In [ ]:
val dataRoot="/datasets/test/raw/mortgage"
val perfPath=dataRoot+"/perf/Performance_2000Q1.txt"
val acqPath=dataRoot+"/acq/Acquisition_2000Q1.txt"
val featureRoot="/datasets/test/mortgage"

import org.apache.spark.storage.StorageLevel

val storageLevel = StorageLevel.MEMORY_ONLY

var year = 0
var quater = 0
for ( year <- 2000 to 2016 ) {
    for ( quater <- 1 to 4) {
        val perfPath = dataRoot + "/perf/Performance_" + year.toString + "Q" + quater.toString + ".txt*"
        val acqPath = dataRoot + "/acq/Acquisition_" + year.toString + "Q" + quater.toString + ".txt*"
        println(perfPath)
        println(acqPath)
        
        val perfCsv = ReadPerformanceCsv(spark, perfPath).persist(storageLevel)
        println(perfCsv.count())
        val acqCsv = ReadAcquisitionCsv(spark, acqPath).persist(storageLevel)
        val perf = CreatePerformanceDelinquency.prepare(perfCsv)
        val cleanDF = CleanAcquisitionPrime(spark, perf, acqCsv).persist(storageLevel)
        val (dfTrain, dfEval) = MortgageXgBoost.transform(cleanDF)
        dfTrain.persist(storageLevel)
        dfEval.persist(storageLevel)
        val parquetDir=featureRoot + "/parquet/" + year.toString + "Q" + quater.toString
        dfTrain.write.mode("overwrite").parquet(parquetDir + "/train")
        dfEval.write.mode("overwrite").parquet(parquetDir + "/eval")
        
        
        val csvDir=featureRoot + "/csv/" + year.toString + "Q" + quater.toString
        dfTrain.write.mode("overwrite").option("header", "true").csv(csvDir + "/train")
        dfEval.write.mode("overwrite").option("header", "true").csv(csvDir + "/eval")
        
        perfCsv.unpersist()
        acqCsv.unpersist()
        cleanDF.unpersist()
        dfTrain.unpersist()
        dfEval.unpersist()
    }
}

/datasets/test/raw/mortgage/perf/Performance_2000Q1.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2000Q1.txt*
9094679
/datasets/test/raw/mortgage/perf/Performance_2000Q2.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2000Q2.txt*
8208081
/datasets/test/raw/mortgage/perf/Performance_2000Q3.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2000Q3.txt*
8726405
/datasets/test/raw/mortgage/perf/Performance_2000Q4.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2000Q4.txt*
10161217
/datasets/test/raw/mortgage/perf/Performance_2001Q1.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2001Q1.txt*
15539871
/datasets/test/raw/mortgage/perf/Performance_2001Q2.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2001Q2.txt*
31540064
/datasets/test/raw/mortgage/perf/Performance_2001Q3.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2001Q3.txt*
27947419
/datasets/test/raw/mortgage/perf/Performance_2001Q4.txt*
/datasets/test/raw/mortgage/acq/Acquisition_2001Q4.txt*
37266966
/datasets/test/raw/mortgage